# Feature Engineering

### Imports

In [99]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
import numpy as np
import pandas as pd
from math import pi

In [100]:
df_train = pd.read_pickle('data/train_clean.pkl')
df_train_calender = pd.read_pickle('data/train_calender_clean.pkl')
df_test = pd.read_pickle('data/test_clean.pkl')
df_test_calender = pd.read_pickle('data/test_calender_clean.pkl')

### Get overview

In [104]:
df_train.head()

,warehouse,date,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,user_activity_1,user_activity_2,id
0,Prague_1,2020-12-05,6895.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1722.0,32575.0,Prague_1_2020-12-05
1,Prague_1,2020-12-06,6584.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1688.0,32507.0,Prague_1_2020-12-06
2,Prague_1,2020-12-07,7030.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1696.0,32552.0,Prague_1_2020-12-07
3,Prague_1,2020-12-08,6550.0,NaN,0,0,0,0,0,0,0,0.0,0,0.8,0.0,1681.0,32423.0,Prague_1_2020-12-08
4,Prague_1,2020-12-09,6910.0,NaN,0,0,0,0,0,0,0,0.0,0,0.5,0.0,1704.0,32410.0,Prague_1_2020-12-09


In [105]:
df_train_calender.head()

,date,holiday_name,holiday,shutdown,mini_shutdown,warehouse_limited,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,warehouse
0,2022-03-09,NaN,0,0,0,0,0,1,0,0,0.0,0,0.0,0.0,Prague_1
1,2019-07-03,NaN,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,Prague_1
2,2020-05-18,NaN,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,Prague_1
3,2021-11-09,NaN,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,Prague_1
4,2023-08-02,NaN,0,0,0,0,0,0,0,0,0.0,0,2.5,0.0,Prague_1


In [106]:
df_test.head()

,warehouse,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id
0,Prague_1,2024-03-16,NaN,0,0,0,0,Prague_1_2024-03-16
1,Prague_1,2024-03-17,NaN,0,0,0,0,Prague_1_2024-03-17
2,Prague_1,2024-03-18,NaN,0,0,0,0,Prague_1_2024-03-18
3,Prague_1,2024-03-19,NaN,0,0,0,0,Prague_1_2024-03-19
4,Prague_1,2024-03-20,NaN,0,0,0,0,Prague_1_2024-03-20


In [107]:
df_test_calender.head()

,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,warehouse
0,2024-11-17,Den boje za svobodu a demokracii,1,0,0,0,Prague_1
1,2024-07-04,NaN,0,0,0,0,Prague_1
2,2024-08-04,NaN,0,0,0,0,Prague_1
3,2024-12-01,NaN,0,0,0,0,Prague_1
4,2024-07-24,NaN,0,0,0,0,Prague_1


### Preprocessing

#### Delete unnecessary columns and save test ids to 'test_ids'

In [108]:
df_train_essence = df_train.drop(columns=['shutdown', 'mini_shutdown', 'blackout', 'mov_change', 'frankfurt_shutdown', 'precipitation', 'snow', 'user_activity_1', 'user_activity_2', 'id'])

In [109]:
df_train_calender_essence = df_train_calender.drop(columns=['shutdown', 'mini_shutdown', 'warehouse_limited', 'blackout', 'mov_change', 'frankfurt_shutdown', 'precipitation', 'snow'])

In [110]:
# df_test_essence = df_test.drop(columns=['id'])
# test_ids = df_test['id'].copy()
df_test_essence = df_test

In [111]:
df_test_calender_essence = df_test_calender

#### Combine train and test with their calenders respectively & add unrepresented dates

In [112]:
# Once for train
df_train_essence.shape

(7340, 8)

In [113]:
df_train_calender_essence.shape

(13307, 7)

In [114]:
# For training data
# Identify common columns, ensuring 'date' and 'warehouse' are included
common_cols = list(set(df_train_essence.columns) & set(df_train_calender_essence.columns))
if 'date' not in common_cols:
    common_cols.append('date')
if 'warehouse' not in common_cols:
    common_cols.append('warehouse')

# Identify columns unique to calendar data
calendar_unique_cols = list(set(df_train_calender_essence.columns) - set(df_train_essence.columns))

# Merge on common columns, including date and warehouse
df_train_combined = pd.merge(df_train_essence, 
                             df_train_calender_essence[common_cols + calendar_unique_cols],
                             on=common_cols,
                             how='outer')

# For test data
# Repeat the process
common_cols_test = list(set(df_test_essence.columns) & set(df_test_calender_essence.columns))
if 'date' not in common_cols_test:
    common_cols_test.append('date')
if 'warehouse' not in common_cols_test:
    common_cols_test.append('warehouse')

calendar_unique_cols_test = list(set(df_test_calender_essence.columns) - set(df_test_essence.columns))

df_test_combined = pd.merge(df_test_essence, 
                            df_test_calender_essence[common_cols_test + calendar_unique_cols_test],
                            on=common_cols_test,
                            how='outer')

In [115]:
df_train_combined['orders'].isna().sum()

5967

In [116]:
df_train_combined.shape

(13307, 8)

In [117]:
df_test_combined.shape

(1869, 8)

In [118]:
df_test_combined.head()

,warehouse,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id
0,Prague_1,2024-03-16,NaN,0,0,0,0,Prague_1_2024-03-16
1,Prague_1,2024-03-17,NaN,0,0,0,0,Prague_1_2024-03-17
2,Prague_1,2024-03-18,NaN,0,0,0,0,Prague_1_2024-03-18
3,Prague_1,2024-03-19,NaN,0,0,0,0,Prague_1_2024-03-19
4,Prague_1,2024-03-20,NaN,0,0,0,0,Prague_1_2024-03-20


#### Combine df_train and df_test into df_combined

In [119]:
# Save use for later
df_train_combined['original_index'] = np.arange(len(df_train_combined))
df_test_combined['original_index'] = np.arange(len(df_test_combined))

df_train_combined['data_source'] = 'train'
df_test_combined['data_source'] = 'test'

df_combined = pd.concat([df_train_combined, df_test_combined], sort=False)

In [120]:
df_combined.head(20647)

,warehouse,date,orders,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,original_index,data_source,id
0,Prague_1,2020-12-05,6895.0,NaN,0,0,0,0,0,train,NaN
1,Prague_1,2020-12-06,6584.0,NaN,0,0,0,0,1,train,NaN
2,Prague_1,2020-12-07,7030.0,NaN,0,0,0,0,2,train,NaN
3,Prague_1,2020-12-08,6550.0,NaN,0,0,0,0,3,train,NaN
4,Prague_1,2020-12-09,6910.0,NaN,0,0,0,0,4,train,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1864,Budapest_1,2024-09-07,NaN,NaN,0,0,0,0,1864,test,NaN
1865,Budapest_1,2024-08-16,NaN,NaN,0,0,0,0,1865,test,NaN
1866,Budapest_1,2024-08-20,NaN,State Foundation Day,1,1,0,0,1866,test,NaN
1867,Budapest_1,2024-05-21,NaN,National Defense Day,0,0,0,0,1867,test,NaN


In [121]:
df_combined.shape

(15176, 11)

In [122]:
df_combined['orders'].isna().sum()

7836

In [123]:
15176-7340

7836

#### Process date column

In [124]:
# Make some more date features
df_combined['date'] = pd.to_datetime(df_combined['date'], errors='coerce')

# Time-based features
df_combined['date_year'] = df_combined['date'].dt.year.fillna(-1)
df_combined['date_month'] = df_combined['date'].dt.month.fillna(-1)
df_combined['date_day'] = df_combined['date'].dt.day.fillna(-1)
df_combined['date_day_of_week'] = df_combined['date'].dt.dayofweek.fillna(-1)
df_combined['date_week_of_year'] = df_combined['date'].dt.isocalendar().week.fillna(-1)
df_combined['date_day_of_year'] = df_combined['date'].dt.dayofyear.fillna(-1)
df_combined['date_quarter'] = df_combined['date'].dt.quarter.fillna(-1)
df_combined['date_is_month_start'] = df_combined['date'].dt.is_month_start.astype(int).fillna(-1)
df_combined['date_is_month_end'] = df_combined['date'].dt.is_month_end.astype(int).fillna(-1)
df_combined['date_is_quarter_start'] = df_combined['date'].dt.is_quarter_start.astype(int).fillna(-1)
df_combined['date_is_quarter_end'] = df_combined['date'].dt.is_quarter_end.astype(int).fillna(-1)

In [125]:
df_combined.head()

,warehouse,date,orders,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,original_index,data_source,...,date_month,date_day,date_day_of_week,date_week_of_year,date_day_of_year,date_quarter,date_is_month_start,date_is_month_end,date_is_quarter_start,date_is_quarter_end
0,Prague_1,2020-12-05,6895.0,NaN,0,0,0,0,0,train,...,12,5,5,49,340,4,0,0,0,0
1,Prague_1,2020-12-06,6584.0,NaN,0,0,0,0,1,train,...,12,6,6,49,341,4,0,0,0,0
2,Prague_1,2020-12-07,7030.0,NaN,0,0,0,0,2,train,...,12,7,0,50,342,4,0,0,0,0
3,Prague_1,2020-12-08,6550.0,NaN,0,0,0,0,3,train,...,12,8,1,50,343,4,0,0,0,0
4,Prague_1,2020-12-09,6910.0,NaN,0,0,0,0,4,train,...,12,9,2,50,344,4,0,0,0,0


In [126]:
df_combined['month_sin'] = np.sin(2 * pi * df_combined['date_month'] / 12)
df_combined['month_cos'] = np.cos(2 * pi * df_combined['date_month'] / 12)
df_combined['day_sin'] = np.sin(2 * pi * df_combined['date_day'] / 31)
df_combined['day_cos'] = np.cos(2 * pi * df_combined['date_day'] / 31)

### Feature engineering


In [127]:
# Assuming df_combined is your DataFrame and it's sorted by date
df_combined = df_combined.sort_values('date')

# Function to find the nearest date before or after
def find_nearest_date(date, dates, direction='before'):
    if direction == 'before':
        nearest = dates[dates < date].max()
    else:  # 'after'
        nearest = dates[dates > date].min()
    return nearest if pd.notna(nearest) else None

# Create new columns
df_combined['holiday_before'] = np.nan
df_combined['holiday_after'] = np.nan

# Iterate through the DataFrame
for idx, row in df_combined.iterrows():
    warehouse = row['warehouse']
    warehouse_dates = df_combined[df_combined['warehouse'] == warehouse]['date'].unique()
    
    date_before = find_nearest_date(row['date'], warehouse_dates, 'before')
    date_after = find_nearest_date(row['date'], warehouse_dates, 'after')
    
    if date_before is not None:
        df_combined.at[idx, 'holiday_before'] = df_combined[
            (df_combined['date'] == date_before) & 
            (df_combined['warehouse'] == warehouse)
        ]['holiday'].values[0]
    
    if date_after is not None:
        df_combined.at[idx, 'holiday_after'] = df_combined[
            (df_combined['date'] == date_after) & 
            (df_combined['warehouse'] == warehouse)
        ]['holiday'].values[0]

# Fill NaN values if needed
df_combined['holiday_before'] = df_combined['holiday_before'].fillna(0)
df_combined['holiday_after'] = df_combined['holiday_after'].fillna(0)

### Lag and Rolling Window Features (abandoned cause no score improvement)

In [128]:
'''
# Assuming df_combined is your DataFrame and it's sorted by date
df_combined = df_combined.sort_values(['warehouse', 'date'])

# Create a column indicating whether a day is a working day (orders is not NaN)
df_combined['is_working_day'] = df_combined['orders'].notna()
'''

"\n# Assuming df_combined is your DataFrame and it's sorted by date\ndf_combined = df_combined.sort_values(['warehouse', 'date'])\n\n# Create a column indicating whether a day is a working day (orders is not NaN)\ndf_combined['is_working_day'] = df_combined['orders'].notna()\n"

In [129]:
'''
# Function to calculate lag and rolling window features for working days
def calculate_working_day_features(df, lag_days, rolling_days):
    df_feature = df[df['data_source'] == 'train']
    
    # Lag features
    for lag in lag_days:
        df[f'lag_{lag}_working'] = df_feature.groupby('warehouse')['orders'].shift(lag)
    
    # Rolling window features
    for window in rolling_days:
        df[f'rolling_mean_{window}_working'] = df_feature.groupby('warehouse')['orders'].transform(lambda x: x.rolling(window).mean())
    
    # Ensure that these features are only calculated for working days
    for feature in [f'lag_{lag}_working' for lag in lag_days] + [f'rolling_mean_{window}_working' for window in rolling_days]:
        df[feature] = df[feature].where(df['is_working_day'], other=np.nan)
    
    return df
'''

"\n# Function to calculate lag and rolling window features for working days\ndef calculate_working_day_features(df, lag_days, rolling_days):\n    df_feature = df[df['data_source'] == 'train']\n    \n    # Lag features\n    for lag in lag_days:\n        df[f'lag_{lag}_working'] = df_feature.groupby('warehouse')['orders'].shift(lag)\n    \n    # Rolling window features\n    for window in rolling_days:\n        df[f'rolling_mean_{window}_working'] = df_feature.groupby('warehouse')['orders'].transform(lambda x: x.rolling(window).mean())\n    \n    # Ensure that these features are only calculated for working days\n    for feature in [f'lag_{lag}_working' for lag in lag_days] + [f'rolling_mean_{window}_working' for window in rolling_days]:\n        df[feature] = df[feature].where(df['is_working_day'], other=np.nan)\n    \n    return df\n"

In [130]:
'''
# Define the number of working days equivalent to your original periods
lag_days = [63, 168, 364]
rolling_days = [63, 168, 364]

# Calculate the working day features
df_combined = calculate_working_day_features(df_combined, lag_days, rolling_days)
'''

'\n# Define the number of working days equivalent to your original periods\nlag_days = [63, 168, 364]\nrolling_days = [63, 168, 364]\n\n# Calculate the working day features\ndf_combined = calculate_working_day_features(df_combined, lag_days, rolling_days)\n'

In [131]:
'''
# Fill NaN values in lag and rolling window features with -1
lag_features = [f'lag_{lag}_working' for lag in lag_days]
rolling_features = [f'rolling_mean_{window}_working' for window in rolling_days]

df_combined[lag_features + rolling_features] = df_combined[lag_features + rolling_features].fillna(-1)
'''

"\n# Fill NaN values in lag and rolling window features with -1\nlag_features = [f'lag_{lag}_working' for lag in lag_days]\nrolling_features = [f'rolling_mean_{window}_working' for window in rolling_days]\n\ndf_combined[lag_features + rolling_features] = df_combined[lag_features + rolling_features].fillna(-1)\n"

### One Hot Encoding

In [132]:
# One-hot encode 'warehouse' column
warehouse_reshaped = df_combined['warehouse'].values.reshape(-1, 1)
warehouse_onehot = OneHotEncoder(sparse_output=False)
warehouse_onehot_encoded = warehouse_onehot.fit_transform(warehouse_reshaped)
warehouse_onehot_encoded_df = pd.DataFrame(warehouse_onehot_encoded, columns=warehouse_onehot.get_feature_names_out(['warehouse']))
df_combined = pd.concat([df_combined.reset_index(drop=True), warehouse_onehot_encoded_df], axis=1)

# One-hot encode 'holiday_name' column in training data
df_combined['holiday_name'] = df_combined['holiday_name'].fillna('NaN')
holiday_name_reshaped_train = df_combined['holiday_name'].values.reshape(-1, 1)
holiday_name_onehot = OneHotEncoder(sparse_output=False)
holiday_name_onehot_encoded_train = holiday_name_onehot.fit_transform(holiday_name_reshaped_train)
holiday_name_onehot_encoded_train_df = pd.DataFrame(holiday_name_onehot_encoded_train, columns=holiday_name_onehot.get_feature_names_out(['holiday_name']))
df_combined = pd.concat([df_combined.reset_index(drop=True), holiday_name_onehot_encoded_train_df], axis=1)

In [133]:
#df_combined = df_combined.drop(columns=['date'])

### Formatting

In [134]:
df_combined = df_combined.drop(columns=['holiday_name', 'warehouse'])

In [135]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15176 entries, 0 to 15175
Data columns (total 71 columns):
 #   Column                                                                    Non-Null Count  Dtype         
---  ------                                                                    --------------  -----         
 0   date                                                                      15176 non-null  datetime64[ns]
 1   orders                                                                    7340 non-null   float64       
 2   holiday                                                                   15176 non-null  int64         
 3   shops_closed                                                              15176 non-null  int64         
 4   winter_school_holidays                                                    15176 non-null  int64         
 5   school_holidays                                                           15176 non-null  int64         
 6   origin

In [136]:
df_train_processed = df_combined[df_combined['data_source'] == 'train'].drop(columns=['data_source'])
df_test_processed = df_combined[df_combined['data_source'] == 'test'].drop(columns=['data_source'])
df_test_processed = df_test_processed.sort_values('original_index')

In [137]:
df_train_processed.shape

(13307, 70)

In [138]:
df_test_processed.shape

(1869, 70)

In [139]:
df_train_processed.head()

,date,orders,holiday,shops_closed,winter_school_holidays,school_holidays,original_index,id,date_year,date_month,...,holiday_name_Memorial Day of the Republic,holiday_name_Memorial day of the 1956 Revolution,holiday_name_NaN,holiday_name_National Defense Day,holiday_name_New Years Day,holiday_name_Peace Festival in Augsburg,holiday_name_Reformation Day,holiday_name_State Foundation Day,holiday_name_Whit monday,holiday_name_Whit sunday
0,2019-01-01,NaN,1,1,0,0,9447,NaN,2019,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2019-01-01,NaN,1,1,0,0,10779,NaN,2019,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-01,NaN,1,1,0,0,9848,NaN,2019,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2019-01-01,NaN,1,1,0,0,8425,NaN,2019,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-01,NaN,1,1,0,0,13275,NaN,2019,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [140]:
df_test_processed.head()

,date,orders,holiday,shops_closed,winter_school_holidays,school_holidays,original_index,id,date_year,date_month,...,holiday_name_Memorial Day of the Republic,holiday_name_Memorial day of the 1956 Revolution,holiday_name_NaN,holiday_name_National Defense Day,holiday_name_New Years Day,holiday_name_Peace Festival in Augsburg,holiday_name_Reformation Day,holiday_name_State Foundation Day,holiday_name_Whit monday,holiday_name_Whit sunday
13308,2024-03-16,NaN,0,0,0,0,0,Prague_1_2024-03-16,2024,3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13320,2024-03-17,NaN,0,0,0,0,1,Prague_1_2024-03-17,2024,3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13327,2024-03-18,NaN,0,0,0,0,2,Prague_1_2024-03-18,2024,3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13331,2024-03-19,NaN,0,0,0,0,3,Prague_1_2024-03-19,2024,3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13335,2024-03-20,NaN,0,0,0,0,4,Prague_1_2024-03-20,2024,3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [141]:
df_train_processed = df_train_processed.drop(columns=['original_index', 'id'])
df_test_processed = df_test_processed.drop(columns=['original_index', 'orders'])

In [142]:
# Remove rows where 'orders' is null in the train set
df_train_processed = df_train_processed.dropna(subset=['orders'])

# Remove rows where 'id' is null in the test set
df_test_processed = df_test_processed.dropna(subset=['id'])

In [143]:
df_train_processed.shape

(7340, 68)

In [144]:
df_test_processed.shape

(397, 68)

### Saving to pkl

In [145]:
df_train_processed.to_pickle('data/train_processed.pkl')

In [146]:
df_test_processed.to_pickle('data/test_processed.pkl')